# Downloader and playlist creator of songs from Tracks

## Step 1 
Run `tracks-downloader.py` to get all tracks from the website. Results in 24148 entries in `tracks-db.txt`.

## Step 2
Run `tracks-db-remove-duplicates.py` to create `tracks-db-no-duplicates.txt` with 6805 entries.

## Step 3
Create an application and get the Client ID and Client Secret codes.
- Head over to Spotify’s developer dashboard and create a new application.
- From the app’s dashboard, click the “Edit Settings” button, and add a Redirect URI to http://localhost:7777/callback.
- Obtain the Client ID and Client Secret codes.

## Step 4
Get OAuth Token from Spotify, according to these guidelines:
- [Bored of Libraries? Here’s How to Connect to the Spotify API Using Pure Python](https://python.plainenglish.io/bored-of-libraries-heres-how-to-connect-to-the-spotify-api-using-pure-python-bd31e9e3d88a) and [SpotifyAPIConnection](https://github.com/SeanDuttonJones/SpotifyAPIConnection)
- [Spotify Authorization Code Flow](https://developer.spotify.com/documentation/general/guides/authorization/code-flow/)

In [1]:
import requests
from urllib.parse import urlencode
import base64

# Enter client id here:
client_id = "" 

# Enter client secret here:
client_secret = ""

auth_headers = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": "http://localhost:7777/callback",
    "scope": "user-library-read"
}

print("Enter this URL into a web browser:")
print("https://accounts.spotify.com/authorize?" + urlencode(auth_headers))

Enter this URL into a web browser:
https://accounts.spotify.com/authorize?client_id=04846d52f69f460ab9787d6ef30c4060&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A7777%2Fcallback&scope=user-library-read


In [2]:
# The enter the URL the browser was redirected to here:
new_url = ""

code = new_url.split("?code=")[1]

encoded_credentials = base64.b64encode(client_id.encode() + b':' + client_secret.encode()).decode("utf-8")

token_headers = {
    "Authorization": "Basic " + encoded_credentials,
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": code,
    "redirect_uri": "http://localhost:7777/callback"
}

r = requests.post("https://accounts.spotify.com/api/token", data=token_data, headers=token_headers)
token_etc = r.json()
token = token_etc["access_token"]

for key, value in token_etc.items():
    print(key, ':\n  ', value)

access_token :
   BQDGi-FNqYBbwF8Jfvcdr3LtCDdcZqppFs4jB-nyZj8uWQrRKPt95uI8gF9EBF8wYE5KJmyGFkmi1V4CCABjvVQkPcsqQTdFM6pAjq26rlOwGrkVIB9ja-SgZh-BSzPcZAJ-46DuYL_YbgLbZwiAQpUi_xKp7L66sMbl5zZG8ycCea9dk-CVdBOP_ASpqtW5WUvW9xuRxMYgNVDIV3M
token_type :
   Bearer
expires_in :
   3600
refresh_token :
   AQCU3OGMIuDMSH3WNqzBqSFFRAQKGjGE_aSlMZEpZrMN3yPGC9mgvafE6BKu8KzrneM9XMtEjHNLRqdtTh-y0VSFT9sR2ERGvIYIuGGffMQjqqy_fjSVAq-M6mQTqwe-zH4
scope :
   user-library-read


## Step 5
Try a search...

In [3]:
user_headers = {
    "Authorization": "Bearer " + token,
    "Content-Type": "application/json",
    "Accept": "application/json"
}

user_params = {
    "limit": 10,
    "q": "artist:MO-DO track:EINS, ZWEI, POLIZEI",
    "type": "track"
}

r = requests.get("https://api.spotify.com/v1/search", params=user_params, headers=user_headers).json()

...and explore the result...

In [47]:
print(type(r))
print(r.keys())

<class 'dict'>
dict_keys(['tracks'])


In [48]:
print(r['tracks'].keys())

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])


In [49]:
print(r['tracks']['href'])
print(r['tracks']['limit'])
print(r['tracks']['next'])
print(r['tracks']['offset'])
print(r['tracks']['previous'])
print(r['tracks']['total'])

https://api.spotify.com/v1/search?query=artist%3AMO-DO+track%3AEINS%2C+ZWEI%2C+POLIZEI&type=track&offset=0&limit=10
10
https://api.spotify.com/v1/search?query=artist%3AMO-DO+track%3AEINS%2C+ZWEI%2C+POLIZEI&type=track&offset=10&limit=10
0
None
29


In [50]:
print(type(r['tracks']['items']))
print(len(r['tracks']['items']))

<class 'list'>
10


In [51]:
print(type(r['tracks']['items'][0]))
print(r['tracks']['items'][0].keys())

<class 'dict'>
dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])


In [52]:
print(r['tracks']['items'][0]['album'])

{'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Y42QTbwNJeqEVmrMknG0f'}, 'href': 'https://api.spotify.com/v1/artists/3Y42QTbwNJeqEVmrMknG0f', 'id': '3Y42QTbwNJeqEVmrMknG0f', 'name': 'Mo-Do', 'type': 'artist', 'uri': 'spotify:artist:3Y42QTbwNJeqEVmrMknG0f'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AO', 'AR', 'AT', 'AU', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LU', 'LY', 'MA', 'MC', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'MT', 'MU', 'MV', 'MW', 'MX', 'MY', 'MZ', '

In [53]:
print(r['tracks']['items'][0]['artists'])

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Y42QTbwNJeqEVmrMknG0f'}, 'href': 'https://api.spotify.com/v1/artists/3Y42QTbwNJeqEVmrMknG0f', 'id': '3Y42QTbwNJeqEVmrMknG0f', 'name': 'Mo-Do', 'type': 'artist', 'uri': 'spotify:artist:3Y42QTbwNJeqEVmrMknG0f'}]


In [54]:
print(r['tracks']['items'][0]['disc_number'])

1


In [55]:
print(r['tracks']['items'][0]['duration_ms'])

313426


In [56]:
print(r['tracks']['items'][0]['explicit'])

False


In [57]:
print(r['tracks']['items'][0]['external_ids'])

{'isrc': 'IT00G9503301'}


In [58]:
print(r['tracks']['items'][0]['external_urls'])

{'spotify': 'https://open.spotify.com/track/015p8TgFahkz0viuux9YGJ'}


In [59]:
print(r['tracks']['items'][0]['href'])

https://api.spotify.com/v1/tracks/015p8TgFahkz0viuux9YGJ


In [60]:
print(r['tracks']['items'][0]['id'])

015p8TgFahkz0viuux9YGJ


In [61]:
print(r['tracks']['items'][0]['is_local'])

False


In [62]:
print(r['tracks']['items'][0]['name'])

Eins Zwei Polizei


In [63]:
print(r['tracks']['items'][0]['popularity'])

58


In [75]:
print(r['tracks']['items'][3]['preview_url'])

https://p.scdn.co/mp3-preview/16407b38bda71015ed98fecb9c23be4198b76f94?cid=04846d52f69f460ab9787d6ef30c4060


In [65]:
print(r['tracks']['items'][0]['track_number'])

1


In [66]:
print(r['tracks']['items'][0]['type'])

track


In [67]:
print(r['tracks']['items'][0]['uri'])

spotify:track:015p8TgFahkz0viuux9YGJ


In [73]:
print(r['tracks']['items'][3]['album']['release_date'])

1994-01-01


In [74]:
print(r['tracks']['items'][3]['album']['album_type'])

single
